# Interatomic Potential Repository Content Manager

This is a working Notebook providing a single location for adding and modifying content in the potentials.nist.gov database.

Notebook outline:

0. Setup: Python library imports and defining global parameters.

1. Citations: Load, add and modify citation information associated with potential models. Each potential should have at least one citation, with the first being the "primary" one for generating IDs. For unpublished potentials, use the unpublished citation option.

2. Potentials: Load, add and modify potential information.  A "potential" should be a unique parameterization with an associated citation. Note that saving to the database is done in step #4.

3. Implementations: Load, add and modify the implementations of a potential. An "implementation" is a specific version/representation of a potential in a specific format. Each potential can have zero, one or multiple implementations.

4. Save potentials: Review the new/modified potentials and save the content to the database.

5. Actions: Actions are used to list changes to the repository. If a potential is added/modified, a corresponding action should also be added.

6. Requests: A request is listed on the repository whenever someone asks for a model that we do not have.

## 0. Setup

### Library imports

In [1]:
import uuid
import datetime
import shutil
from pathlib import Path

import potentials

import numpy as np

# Jupyter display libraries
from IPython.core.display import display, HTML

### Global parameters

- __complete_potentials__ is a compiled list of the potentials whose addition/changes were uploaded to the database.  This is used by step #5 for reporting website changes.  This should be reset after creating an Action for meaningful sets of potentials.

- __db__ is the Database object for accessing and uploading records to the database.  __Note__: you must have an account on potentials.nist.gov and sign in to be able to add or modify content.

In [2]:
complete_potentials = []

In [3]:
db = potentials.Database(username='lmh1')

Enter password for lmh1 @ https://potentials.nist.gov:········


- - -

## 1. Citations

In [4]:
db.load_citations(verbose=True)

Loaded 276 remote citations


### 1.1 Build or load citation

#### Option #1: For existing citations and/or new dois

In [61]:
doi = '10.1088/0965-0393/24/8/085017'

citation = db.get_citation(doi, verbose=True)

Citation retrieved from loaded citations


#### Option #2: For new doi-less citations

In [5]:
author = 'Bob Test'
year = 2099
ID = note = f'{year}--Test-L-M'

citation = potentials.Citation(ENTRYTYPE = 'unpublished',
                               title = 'to be published',
                               author = author,
                               ID = ID,
                               note = note,
                               year = year)

### 1.2 Investigate and modify

- Add abstract

- Check and replace garbled latex or other symbols in authors, title and abstract

- Verify correct year and page number(s)

#### 1.2.1 Generate html and list content fields

In [62]:
display(HTML(citation.html()))
print()
for key, value in citation.asdict().items():
    print(key+':', value)


ENTRYTYPE: article
ID: Borovikov_2016
abstract: Dislocation nucleation from grain boundaries (GB) can control plastic deformation in nano-crystalline metals under certain conditions, but little is known about what controls dislocation nucleation, because when data from different materials are compared, the variations of many interacting properties tend to obscure the effects of any single property. In this study, we seek clarification by applying a unique capability of semi-empirical potentials in molecular dynamics simulations: the potentials can be modified such that all significant material properties but one, are kept constant. Using a set of potentials developed to isolate the effects of stacking fault energy, we show that for a given grain boundary, loading orientation and strain rate, the yield stress depends linearly on both the stable and unstable stacking fault energies. The coefficients of proportionality depend on the GB structure and the value of the yield stress is relat

#### 1.2.2 Modify and update fields

Any valid bibtex fields can be added, but citation rendering relies only on the primary fields. 

In [36]:
citation.abstract = ' '.join(['In this atomistic study on the copper–nickel system, a new embedded-atom alloy potential',
                              'between copper and nickel is fitted to experimental data on the mixing enthalpy, taking',
                              'available potentials for the pure components from literature. The resulting phase boundaries',
                              'of the new potential are in very good agreement with a recent CALPHAD prediction. Using',
                              'this new potential, a high angle symmetrical tilt Σ5 and a coherent Σ3 twin grain boundary',
                              '(GB) are chosen for a systematic investigation of equilibrium GB segregation in the',
                              'semi-grandcanonical ensemble at temperatures from 400 K to 800 K. Applying thermodynamically',
                              'accurate integration techniques, the GB formation energies are calculated exactly and as an',
                              'absolute value for every temperature and composition, which also enables the evaluation of',
                              'GB excess entropies. The thorough thermodynamic model of GBs developed by Frolov and Mishin',
                              'is excellently confirmed by the simulations quantitatively, if the impact of both segregation',
                              'and GB tension on the change in GB formation energy is accounted for. In the case of the',
                              'Σ3 coherent GB, it turns out that the change in GB formation energy at low temperatures is',
                              'for the most part attributed to the GB tension, while segregation only has a small',
                              'influence. This demonstrated effect of GB tensions should also be taken into account in the',
                              'interpretation of experiments.'])
#citation.author
#citation.volume
#citation.number
#citation.title = 'Water Modeled As an Intermediate Element between Carbon and Silicon'
#citation.year
#citation.pages

#### 1.2.3 Review and Save Citation

In [ ]:
display(HTML(citation.html()))

In [38]:
db.save_citation(citation, verbose=True)

record 10.1016_j.actamat.2019.06.027 (5e5ea47cab2f7c002e01110d) successfully uploaded.
Citation added to database


- - -

## 2. Potentials

In [9]:
db.load_potentials(verbose=True)

Loaded 289 remote potentials


### 2.1 Build or load potential

#### Option #1: Build new potential using citation(s)

- __elements__ (*list*) The element models used by the potential.
- __citations__ (*list*) The citation(s) to associate with the potential.
- __notes__ (*str, optional*) Notes providing more information on the potential, such as usage notes and how the potential relates to other potentials.
- __key__ (*str, optional*) UUID4 key to assign to the potential.  If None, a new key will be generated.
- __othername__ (*str, optional*) Specifies an alternate description of the interactions besides the list of elements.  Examples include if the model is "universal", designed for a specific compound, or is a coarse-grain representation.
- __fictional__ (*bool, optional*) Flag indicating if the model is "fictional", i.e. purposefully fit to unrealistic properties. 
- __modelname__ (*str, optional*) Used if citation + element info is not enough to create a unique ID, e.g. a publication lists multiple parameterizations of the same element. The value should make it clear which version it is associated with, and correspond to how the authors refer to it when possible.
- __recorddate__ (*datetime.date, optional*) Date to assign to the record.  If not given, will use today's date.

In [112]:
potential = potentials.Potential(
    elements=['Cu'],
    citations=[citation],
    #notes=None,
    #key=None,
    #othername='water',
    fictional=True,
    modelname='34',
    #recorddate=datetime.date(2019, 12, 24)
    )

#### Option #2: Select existing potential

In [46]:
potential = db.get_potential(id='2015--Broqvist-P-Kullgren-J-Wolf-M-J-et-al--Ce-O')

### 2.2 Investigate and modify

#### 2.2.1 Generate html and list content fields

In [47]:
display(HTML(potential.html()))
print()
for key, value in potential.asdict().items():
    print(key+':', value)


key: 898da38c-c94b-4254-b480-35f3b1ab22ac
id: 2015--Broqvist-P-Kullgren-J-Wolf-M-J-et-al--Ce-O
recorddate: 2018-08-15
notes: J. Kullgren included the following notes: "Usage: The parameters have been tested for static calculations of CeO2 and partially reduced CeO(2-x) using the LAMMPS code with the fortran implementation of reaxFF. For energy comparisons, use the in-cell approach (see the paper) when calculating reaction energies. Note to the users: After publication we have made further use of the published ceria parameters and noticed an additional (false) local minimum occurring for partially reduced ceria at a short Ce-O distance (approx. 1.89 Angstrom). This may (for example) have consequences for dynamic simulations at moderate temperatures. Our attempts to heal this deficiency have so far destroyed the good performance regarding the ordering of the surface vacancy energies on the (111) surface. In relevant cases, we advice our users to analyze the bond distances from the simul

#### 2.2.2 Modify and update fields

Same fields as above

In [22]:
#potential.key = 
#potential.recorddate = datetime.date()
#potential.elements = []
#potential.othername = 'water'
#potential.fictional = 
#potential.modelname = 

In [113]:
potential.notes = ' '.join(["This listing is for the MCu34 parameterization listed in the reference.",
                            "Dr. M.I. Mendelev (Ames Laboratory) noted that these are new fictional potentials intended",
                            "to supplement the existing potentials posted to the NIST repository",
                            "(as the 2015--Borovikov-V-Mendelev-M-I-King-A-H-LeSar-R--fictional-Cu-# listings).",
                            'Dr. Mendelev further noted that, "the new potentials provide the same SFE as',
                            '<a href="https://www.ctcms.nist.gov/potentials/entry/2013--Mendelev-M-I-King-A-H--Cu/">2013--Mendelev-M-I-King-A-H--Cu</a>',
                            'but different unstable stacking fault',
                            'energy (USFE). All these Cu fictional potentials are designed to study the effect of SFE and USFE on the',
                            'deformation behavior in fcc metals."  Reference information added March 5, 2020.'])         

In [ ]:
#potential.notes += 

#### 2.2.3 Delete potential (PERMANENT!)

In [146]:
###db.cdcs.delete_record(title='potential.'+potential.id, template='Potential')

record potential.2015--Mendelev-M-I--fictional-Mg (5df2accf90985100269d7b43) has been deleted.


## 3. Implementations

### 3.1 List all current implementation types

List is given to help keep type values (relatively) uniform. 

In [150]:
imptypes = []
for pot in db.potentials:
    for imp in pot.implementations:
        imptypes.append(imp.type)
imptypes = np.unique(imptypes)

print('All existing implementation types:')
for imptypes in imptypes:
    print('   ', imptypes)

All existing implementation types:
    ADP tabulated functions
    Dynamo MEAM
    EAM setfl
    EAM tabulated functions
    Equations
    FORTRAN
    Finnis-Sinclair tables
    GULP
    IMD option EAM
    LAMMPS pair_style adp
    LAMMPS pair_style agni
    LAMMPS pair_style bop
    LAMMPS pair_style comb3
    LAMMPS pair_style eam
    LAMMPS pair_style eam/alloy
    LAMMPS pair_style eam/cd
    LAMMPS pair_style eam/fs
    LAMMPS pair_style edip
    LAMMPS pair_style edip/multi
    LAMMPS pair_style eim
    LAMMPS pair_style extep
    LAMMPS pair_style hybrid/overlay eam/alloy eam/fs
    LAMMPS pair_style hybrid/overlay zbl eam/alloy
    LAMMPS pair_style hybrid/overlay zbl snap
    LAMMPS pair_style lcbop
    LAMMPS pair_style meam
    LAMMPS pair_style meam (modified)
    LAMMPS pair_style meam/spline
    LAMMPS pair_style polymorphic
    LAMMPS pair_style sw
    LAMMPS pair_style tersoff
    LAMMPS pair_style tersoff/mod
    LAMMPS pair_style tersoff/mod/c
    LAMMPS pair_style te

### 3.2 Build or load implementation

#### 3.2.1 List ids for current potential and implementations

In [12]:
print('potential id:', potential.id)
print('implementation ids:')
for i, implementation in enumerate(potential.implementations):
    print(f'{i}: {implementation.id}')

potential id: 1990--Ackland-G-J-Vitek-V--Cu
implementation ids:
0: 1990--Ackland-G-J--Cu--LAMMPS--ipr1
1: 1990--Ackland-G-J--Cu--LAMMPS--ipr2


#### 3.2.2 Build new implementation

- __type__ (*str*) Describes the type of the implementation. Use one of the listed values above if possible.
- __id__ (*str*) Unique human-readable id for the implementation. For new content, should be derived from the associated potential id:
    - Remove all authors except for the first
    - Add a simple format descriptor: LAMMPS, GULP, table, parameters, FORTRAN, etc.
    - Add a version descriptor: ipr1, ipr2, etc
- __notes__ (*str, optional*) Notes on the implementation.  This includes where the file(s) came from, who created them, testing info, how it differs from other versions, etc.
- __key__ (*str, optional*) UUID4 key to assign to the implementation.  If None, a new key will be generated.
- __status__ (*str, optional*) Indicates the status of the implementation. Available values are 
    - "active" (default) indicates a current implementation.
    - "superseded" indicates an implementation that is still consistent with the potential model, but has minor issues that were fixed by a newer implementation.
    - "retracted" indicates an implementation that was identified as being an invalid representation of the potential model.
- __date__ (*datetime.date, optional*) The date that the implementation was submitted or added.  If not given, will use today's date.

In [151]:
implementation = potentials.Implementation(
    type='LAMMPS pair_style reax/c',
    id='2015--Broqvist-P--Ce-O--LAMMPS--ipr2',
    notes=' '.join(['These files were posted on March 15, 2020 by Lucas Hale. By separating the comments into a separate file, this parameter file is compatible with LAMMPS.']),
    key='e5ed3ebc-d8bc-49fd-856d-266eb708f269',
    #status=None,
    date=datetime.date(2020, 3, 15), 
)
index = len(potential.implementations)
potential.implementations.append(implementation)

#### 3.2.3 Select implementation

Must be done

In [51]:
index = 1
implementation = potential.implementations[index]

### 3.3 Modify implementation

#### 3.3.1 Modify implementation metadata 

In [52]:
display(HTML(implementation.html()))

In [8]:
#implementation.type='LAMMPS pair_style sw',
#implementation.key=None,
#implementation.id='2009--Molinero-V--water--ipr-1',
#implementation.status=None,
#implementation.date=datetime.date(2020, 1, 10), 
implementation.notes= ' '.join(['The parameter file mW.sw was provided by Rodrigo Freitas (Standford) on Jan 10, 2020.',
                                'main.pdf contains computed properties and references that show this LAMMPS implementation',
                                'to give predictions consistent with what is reported in the original paper.',
                                ])

In [191]:
implementation.date=datetime.date(2014, 11, 24)

In [188]:
implementation.notes = 'This file was provided by Sergei Starikov (Ruhr-Universität Bochum, Germany) on 5 May 2019. It has been carefully tested and gives the expected property predictions. Update March 15, 2020: This version was identified to not be compatible with LAMMPS versions after 7 Aug 2019 due to more rigorous format checks.'

In [135]:
implementation.notes += ' Update Jan 15, 2020: It was noticed that the original file hosted here was truncated and incomplete.'
implementation.notes += ' The incomplete file will not work with LAMMPS versions after 7 Aug 2019.'
implementation.notes += ' For earlier LAMMPS versions, both versions of the parameter file appear to behave identically.'

In [50]:
implementation.status = 'retracted'
implementation.notes = 'This file was sent by J. Kullgren (Uppsala University) on 19 December 2016 and posted with his permission. Update March 15, 2020: This version was identified to not be compatible with LAMMPS.'

In [53]:
implementation.notes='These files were posted on March 15, 2020 by Lucas Hale. They modify the above version by separating the comments into a separate file, making the parameter file compatible with LAMMPS.'

In [162]:
implementation.type = 'LAMMPS pair_style reax/c'

In [203]:
implementation.id = '2013--Bonny-G--Fe-Cr-W--LAMMPS--ipr1'

#### 3.3.2 Add implementation content

The content for the implementation can exist in one of the following formats

1. Artifacts: files that are hosted on the potentials repository.
2. Parameters: list of the potential's parameters to show as html text on the repository.
3. WebLinks: hyperlinks to content hosted externally.

#### Option #1: Add artifacts

Each artifact has:

- __filename__ (*str*) The name of the file (no path information).
- __url__ (*str*) The absolute url where the file can be downloaded from.
- __label__ (*str, optional*) An optional descriptive label for the file.

In [153]:
# Define paths to files
newpotpath = Path('C:/Users/lmh1/Documents/website/new potentials')
webpath = 'https://www.ctcms.nist.gov/potentials/'
localpath = Path('C:/Users/lmh1/Documents/website/IPR-website/potentials/')
relpath = f'Download/{potential.id}/{index+1}/'

In [154]:
# Specify files from newpotpath to add
filenames = [
    Path(newpotpath, 'ffield_Ce-O'),
    Path(newpotpath, 'comments.txt'),
    ]

labels = [None for i in range(len(filenames))]

In [122]:
# Specify files from newpotpath to add
filenames = [
    Path(newpotpath, 'READ_ME.txt'),
    Path(newpotpath, 'FeCrW_d.eam.alloy'),
    Path(newpotpath, 'FeCrW_s.eam.fs'),
    ]

labels = [
    'Documentation',
    'd_band',
    's_band',
]

In [155]:
# Make download directory in local copy of website
downloadpath = Path(localpath, relpath)
if not downloadpath.is_dir():
    downloadpath.mkdir(parents=True)

# Copy files and add artifact listings
for filename, label in zip(filenames, labels):
    # Build url
    url = webpath + relpath + filename.name
    # Copy files
    shutil.copy(filename, downloadpath)
    print(Path(downloadpath, filename.name))
    
    # Add artifact listing to implementation
    implementation.add_artifact(filename=filename.name, url=url, label=label)

C:\Users\lmh1\Documents\website\IPR-website\potentials\Download\2015--Broqvist-P-Kullgren-J-Wolf-M-J-et-al--Ce-O\2\ffield_Ce-O
C:\Users\lmh1\Documents\website\IPR-website\potentials\Download\2015--Broqvist-P-Kullgren-J-Wolf-M-J-et-al--Ce-O\2\comments.txt


In [20]:
implementation.artifacts[0].url

'https://www.ctcms.nist.gov/potentials/Download/1987--Ackland-G-J-Thetford-R--Mo/4/mo.fs.eam.alloy'

In [151]:
implementation.artifacts[0].url = 'https://www.ctcms.nist.gov/potentials/Download/2016--Wilson-S-R-Mendelev-M-I--fictional-W/1/W2.eam.fs'

#### Option #2: Add parameters

Each parameter has:

- __name__ (*str, optional*) The name of the parameter or string parameter line.
- __value__ (*float, optional*) The value of the parameter.
- __unit__ (*str, optional*) Units associated with value.

Note: if the parameters are purely text-based, you can use only the name field.

#### Option #3: Add weblinks

Each weblink has:

- __url__ (*str*) URL for the link.
- __label__ (*str, optional*) A short description label to proceed the link.
- __linktext__ (*str, optional*) The text for the link, i.e. what gets clicked on.

## 4. Review and save potentials

In [54]:
display(HTML(potential.html()))

In [55]:
db.upload_potential(potential)
complete_potentials.append(potential)

record potential.2015--Broqvist-P-Kullgren-J-Wolf-M-J-et-al--Ce-O (5df2accd90985100329d7afd) has been updated.


## 5. potential_LAMMPS

This section generates the potential_LAMMPS metadata records for generating the LAMMPS input command lines for supported potentials.

### 5.1 Common settings

- __id__ (*str, optional*) implementation id. Should match with the implementation id from above.
- __key__ (*str, optional*) implementation key. Should match with the implementation key from above.
- __potid__ (*str, optional*) potential id. Should match with the potential id from above.
- __potkey__ (*str, optional*) potential key. Should match with the potential key from above.
- __elements__ (*list, optional*) The elements that the implementation simulates.  Should match with the elements from above if the implementation explicitly models each element.
- __symbols__ (*list, optional*) The particle model symbols defined by the implementation.  Not needed if identical to elements.
- __masses__ (*list, optional*) The masses to assign to each symbol/element.  Should be given if the implementation specifies the mass.  Required if elements is not given.
- __units__ (*str, optional*) The LAMMPS units to use with the implementation.  Default value is 'metal'.
- __atom_style__ (*str, optional*) The LAMMPS atom_style to use with the implementation.  Default value is 'atomic'.
- __pair_style__ (*str*) The LAMMPS pair_style associated with the implementation.
- __pair_style_terms__ (*list, optional*) Any additional terms that should appear on the pair_style command line.

In [121]:
kwargs = {}
kwargs['id'] = implementation.id
kwargs['key'] = implementation.key
kwargs['potid'] = potential.id
kwargs['potkey'] = potential.key

kwargs['elements'] = potential.elements
#kwargs['symbols'] = 
kwargs['masses'] = [63.546]

#kwargs['units'] =
#kwargs['atom_style'] = 

kwargs['pair_style'] = 'eam/fs'
#kwargs['pair_style_terms'] = ['polar_off']

### 5.2 Style-specific settings

#### Option #1 For pair potentials without parameter files, e.g lj/cut

In [ ]:
kwargs['interactions'] = [
    {'symbols': ['Al', 'Al'], 'terms': [1.23, 3.412]},
    {'symbols': ['Al', 'Cu'], 'terms': [1.23, 3.412]},
    {'symbols': ['Cu', 'Cu'], 'terms': [1.23, 3.412]},
]

#### Option #2 For potentials with single parameter files, e.g. eam/alloy

In [122]:
kwargs['paramfile'] = 'MCu34.eam.fs'

#### Option #3 For potentials with a parameter file and a library file, e.g. meam

In [24]:
kwargs['libfile'] = 'library.meam'
kwargs['paramfile'] = 'parameter.meam'

#### Option #4 For classic EAM potentials, i.e. eam

In [ ]:
kwargs['paramfiles'] = []

#### Option #5 For OpenKIM potentials

In [ ]:
kwargs['kimid'] = 

#### Option #999 Read local file 

In [24]:
from DataModelDict import DataModelDict as DM

In [168]:
title = '2015--Broqvist-P--Ce-O--LAMMPS--ipr2'

fdir = Path('C:/Users/lmh1/Documents/website/new potentials/`fixed_pots')
fname = Path(fdir, f'{title}.json')
potential_lammps = DM(fname)

In [169]:
print(potential_lammps.json(indent=2))

{
  "potential-LAMMPS": {
    "key": "e5ed3ebc-d8bc-49fd-856d-266eb708f269", 
    "id": "2015--Broqvist-P--Ce-O--LAMMPS--ipr2", 
    "potential": {
      "key": "898da38c-c94b-4254-b480-35f3b1ab22ac", 
      "id": "2015--Broqvist-P-Kullgren-J-Wolf-M-J-et-al--Ce-O"
    }, 
    "units": "metal", 
    "atom_style": "charge", 
    "atom": [
      {
        "element": "Ce", 
        "mass": 140.1
      }, 
      {
        "element": "O", 
        "mass": 15.999
      }
    ], 
    "pair_style": {
      "type": "reax/c", 
      "term": {
        "option": "NULL"
      }
    }, 
    "pair_coeff": {
      "term": [
        {
          "file": "ffield_Ce-O"
        }, 
        {
          "symbols": "true"
        }
      ]
    }, 
    "command": {
      "term": [
        {
          "option": "fix"
        }, 
        {
          "option": "qeq_fix"
        }, 
        {
          "option": "all"
        }, 
        {
          "option": "qeq/reax"
        }, 
        {
          "parameter": 

In [172]:
template = 'potential_LAMMPS'
content = potential_lammps.xml().replace('True', 'true').replace('False', 'false').replace('1e-06', '0.000001')
db.cdcs.upload_record(content=content, template=template, title=title)

record 2015--Broqvist-P--Ce-O--LAMMPS--ipr2 (5e6eabecab2f7c0033011228) successfully uploaded.


### 5.3 Build, review and save

In [123]:
potential_lammps = potentials.build_potential_LAMMPS(**kwargs).build()
print(potential_lammps.json(indent=2))

{
  "potential-LAMMPS": {
    "key": "ca4f2e15-6668-42ac-aeae-ce02625c1216", 
    "id": "2016--Borovikov-V--fictional-Cu-34--LAMMPS--ipr1", 
    "potential": {
      "key": "a08e16b3-fd6e-471a-8961-a22f92a10ea1", 
      "id": "2016--Borovikov-V-Mendelev-M-I-King-A-H--fictional-Cu-34"
    }, 
    "units": "metal", 
    "atom_style": "metal", 
    "atom": {
      "element": "Cu", 
      "mass": 63.546
    }, 
    "pair_style": {
      "type": "eam/fs"
    }, 
    "pair_coeff": {
      "term": [
        {
          "file": "MCu34.eam.fs"
        }, 
        {
          "symbols": "True"
        }
      ]
    }
  }
}


In [124]:
potdirpath = Path('library/potential_LAMMPS', implementation.id)
potfilepath = Path('library/potential_LAMMPS', implementation.id + '.json')

try:
    potdirpath.mkdir(parents=True)
except:
    pass
for filename in filenames:
    shutil.copy(filename, potdirpath)

with open(potfilepath, 'w') as f:
    potential_lammps.json(fp=f, indent=4)

In [125]:
title = potential_lammps['potential-LAMMPS']['id']
content = potential_lammps.xml().replace('True', 'true').replace('False', 'false')
template = 'potential_LAMMPS'
db.cdcs.upload_record(content=content, template=template, title=title)

record 2016--Borovikov-V--fictional-Cu-34--LAMMPS--ipr1 (5e617f62ab2f7c002601118c) successfully uploaded.


In [ ]:
db.upload_lammps_potential(potential_lammps)

In [92]:
pot = db.get_lammps_potential(id='2015--Broqvist-P--Ce-O--LAMMPS--ipr2')

In [94]:
pot.status

'active'

In [72]:
for pot in pots:
    content = pot.asmodel()
    content['potential-LAMMPS']['status'] = 'retracted'
    newpot = potentials.PotentialLAMMPS(content)
    db.upload_lammps_potential(newpot)    

record 1990--Ackland-G-J--Cu--LAMMPS--ipr1 (5dfa6fdeab2f7c0033824b0a) has been updated.
record 2009--Zhakhovskii-V-V--Al--LAMMPS--ipr1 (5dfa7310ab2f7c0030824ab2) has been updated.
record 2009--Zhakhovskii-V-V--Au--LAMMPS--ipr1 (5dfa7311ab2f7c0034824e0e) has been updated.
{'message': "Element 'parameter': '1e-06' is not a valid value of the atomic type 'xs:decimal'., line 2"}


In [85]:
content = newpot.asmodel()
content['potential-LAMMPS']['command']['term'][-2]['parameter'] = '0.000001'

In [86]:
print(content.json(indent=4))

{
    "potential-LAMMPS": {
        "key": "7fc4dd24-6508-4b1c-8a2c-afd4c3addf23", 
        "id": "2015--Broqvist-P--Ce-O--LAMMPS--ipr1", 
        "status": "retracted", 
        "potential": {
            "key": "898da38c-c94b-4254-b480-35f3b1ab22ac", 
            "id": "2015--Broqvist-P-Kullgren-J-Wolf-M-J-et-al--Ce-O"
        }, 
        "units": "metal", 
        "atom_style": "charge", 
        "atom": [
            {
                "element": "Ce", 
                "mass": 140.1
            }, 
            {
                "element": "O", 
                "mass": 15.999
            }
        ], 
        "pair_style": {
            "type": "reax/c", 
            "term": {
                "option": "NULL"
            }
        }, 
        "pair_coeff": {
            "term": [
                {
                    "file": "ffield_Ce-O.txt"
                }, 
                {
                    "symbols": "true"
                }
            ]
        }, 
        "command": {
  

In [96]:
c = """{
    "potential-LAMMPS": {
        "key": "1cb880e7-0cef-462f-8dde-9e891ada27e5", 
        "id": "2016--Beland-L-K--Ni-Co--LAMMPS--ipr2", 
        "potential": {
            "key": "8435450a-75e2-4c19-885f-ad2bb8ca34e6", 
            "id": "2016--Beland-L-K-Lu-C-Osetskiy-Y-N-et-al--Ni-Co"
        }, 
        "units": "metal", 
        "atom_style": "atomic", 
        "atom": [
            {
                "element": "Ni", 
                "mass": 58.6934
            }, 
            {
                "element": "Co", 
                "mass": 58.9332
            }
        ], 
        "pair_style": {
            "type": "hybrid/overlay",
            "term": {
                "option": "zbl eam/alloy"
            }
        }, 
        "pair_coeff": [
            {
                "interaction": {
                    "symbol": [
                        "Ni", 
                        "Ni"
                    ]
                }, 
                "term": [
                    {
                        "option": "zbl"
                    }, 
                    {
                        "parameter": 28
                    }, 
                    {
                        "parameter": 28
                    }
                ]
            },
            {
                "interaction": {
                    "symbol": [
                        "Ni", 
                        "Co"
                    ]
                }, 
                "term": [
                    {
                        "option": "zbl"
                    }, 
                    {
                        "parameter": 28
                    }, 
                    {
                        "parameter": 27
                    }
                ]
            },
            {
                "interaction": {
                    "symbol": [
                        "Co", 
                        "Co"
                    ]
                }, 
                "term": [
                    {
                        "option": "zbl"
                    }, 
                    {
                        "parameter": 27
                    }, 
                    {
                        "parameter": 27
                    }
                ]
            },
            {
                "term": [
                    {
                        "file": "NiCo-lammps-2014.alloy"
                    }, 
                    {
                        "symbols": "true"
                    }
                ]
            }
        ]
    }
}"""

In [97]:
newpot = potentials.PotentialLAMMPS(c)

In [98]:
db.upload_lammps_potential(newpot)

record 2016--Beland-L-K--Ni-Co--LAMMPS--ipr2 (5e84a7d9ab2f7c0033011234) successfully uploaded.


In [95]:
db.cdcs.delete_record(template='potential_LAMMPS', title='2015--Thompson-A-P--Ta--LAMMPS--ipr2a')

record 2015--Thompson-A-P--Ta--LAMMPS--ipr2a (5dfa7368ab2f7c0030824aba) has been deleted.


- - -

## 5. Actions

Actions are used by the website to note changes in content and representation. 

### 5.1 Build action

#### Option #1: new posting(s)

In [173]:
for complete_potential in complete_potentials:
    print(complete_potential.id)

1987--Ackland-G-J-Thetford-R--Mo
1990--Ackland-G-J-Vitek-V--Cu
2009--Zhakhovskii-V-V-Inogamov-N-A-Petrov-Y-V-et-al--Al
2009--Zhakhovskii-V-V-Inogamov-N-A-Petrov-Y-V-et-al--Au
2011--Bonny-G-Pasianot-R-C-Terentyev-D-Malerba-L--Fe-Cr
2013--Bonny-G-Castin-N-Bullens-J-et-al--Fe-Cr-W
2013--Smirnova-D-E-Kuksin-A-Y-Starikov-S-V-et-al--U-Mo-Xe
2015--Broqvist-P-Kullgren-J-Wolf-M-J-et-al--Ce-O


In [53]:
action = potentials.Action(
    type = 'new posting',
    potentials = complete_potentials,
   # date = None,
    comment = 'New posting for Cu-Ni',
)

#### Option #2: updated posting(s)

In [174]:
action = potentials.Action(
    type = 'updated posting',
    potentials = complete_potentials,
    date = None,
    comment = 'Corrected parameter files for new LAMMPS versions',
)

#### Option #3: retraction

In [ ]:
action = potentials.Action(
    type = 'retraction',
    potentials = complete_potentials,
    date = None,        
    comment =,
)

#### Option #4: site change

In [193]:
action = potentials.Action(
    type = 'site change',
    date = None,
    comment = 'IDs added for all non-LAMMPS implementations. The handling of fictional potentials made consistent with "real" potentials.',
    potentials = []
)

### 5.2 Look and save

In [194]:
display(HTML(action.html()))

In [195]:
title = f"{action.date} {action.comment[:90]}"
content = action.asmodel().xml()
template = 'Action'
db.cdcs.upload_record(content=content, template=template, title=title)

record 2020-03-15 IDs added for all non-LAMMPS implementations. The handling of fictional potentials made co (5e6eb8efab2f7c0030011140) successfully uploaded.


- - -

## 6. Requests

Requests are used by the website to list when people are looking for specific models that we don't have.

### 6.1 Build request

In [63]:
request = potentials.Request(
    #comment = 'reactive potentials',
    date = datetime.date(2020, 1, 2),
    systems = [
        {
            #'formula':,
            'elements': ['Mg', 'Cu'],
        },
        #{
        #    'formula': 'Moose',
       #     'elements': ['Ti', 'Cl'],
       # },
    ]
)

### 6.2 Look and save

In [64]:
display(HTML(request.html()))

In [65]:
model = request.asmodel()
elements = ' '.join(model.finds('element'))

title = f'{request.date} {elements}'
content = model.xml()
template = 'Request'

db.cdcs.upload_record(content=content, template=template, title=title)

record 2020-01-02 Mg Cu (5e5eb358ab2f7c0033011111) successfully uploaded.
